## 1. 패키지 로드

In [32]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm_notebook

In [31]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import itertools

## 2. 데이터 로드

In [2]:
actor_df = pd.read_csv('../text_data/actor_data_keyword.csv', encoding='utf-8-sig')
webtoon_df = pd.read_csv('../text_data/webtoon_data_keyword.csv', encoding='utf-8-sig')

## 3. 데이터 전처리

In [3]:
actor_df.head(2)

,Name,Age,Sex,수상내역,드라마활동,activation_content,사진url,배역내용,"제목, 내용",keywords
0,감우성,52,남,"['2018 SBS 연기대상 대상', '2018 SBS 연기대상 베스트 커플상', ...",['바람이 분다/주연/권도훈 역/https://search.naver.com/sea...,기 공채 탤런트로 배우 생활을 시작했으며 동기로 한석규 박철 차광수 곽진영 김소이...,https://search.pstatic.net/common?type=b&size=...,첫사랑 수진과 열렬한 연애 끝에 결혼까지 성공했다 남에게 피해 주지 않고 올곧고 바...,감우성 닮으면 잘생긴거냐내 친구 미용실가서 누나가 감우성 닮았다는데 잘생긴거냐 정우...,"괜찮은, 예민함이, 있었네요, 작음, 그러겟지, 자유롭게, 당해서, 있나, 비슷해,..."
1,강경준,39,남,"['2017 MBC 연기대상 연속극부문 남자 우수연기상', '2004 MBC 방송연...",['별별 며느리/주연/최한주 역/https://search.naver.com/sea...,초등학교 시절에는 야구 중학교 시절에는 농구선수였었다 농구를 소재로 한 드라마 마지...,https://search.pstatic.net/common?type=b&size=...,흙수저 아니 몸짱 얼짱 마음까지 짱인 태권도장 사범 얼굴만 봐도 신뢰감이 뚝뚝 묻어...,강경준 강 산 강승호네이름 두번이나 바꿨구나그러고보니 남다 딱 장신영강경준이랑 ...,"이뻐, 괜찮은, 예민함이, 있었네요, 그러면서, 쿨한, 작음, 그러겟지, 자유롭게,..."


In [4]:
webtoon_df.head(2)

,Title,Character,Sex,Age,설명,keyword
0,폭풍의전학생,주인공,남,10대,최완서를 얼굴에 니킥을 작렬시켜 한방에 보내버리고 정해인을 박치기 한다음에 머리를 ...,"순전히, 있지만, 엄청난, 태연한, 뛰어난데, 가까운, 약해서, 진정한, 안되지만,..."
1,폭풍의전학생,최완서,남,10대,전 3반짱. 묘한 색깔의 피부[7]와 부서진 콧잔등이 포인트. 전설의 레전드의 최대...,"불쌍한, 멀쩡하던, 수많은, 자리는, 평범하다, 좋아하는, 아니며, 없으며, 강한,..."


### 1) actor data : Name, Age, Sex, activation_content, 배역내용, 제목, 내용, keywords만 가져온 데이터 프레임 생성

In [7]:
actor_df = actor_df[['Name', 'Age', 'Sex', 'activation_content', '배역내용', '제목, 내용', 'keywords']]

In [8]:
actor_df.head(2)

,Name,Age,Sex,activation_content,배역내용,"제목, 내용",keywords
0,감우성,52,남,기 공채 탤런트로 배우 생활을 시작했으며 동기로 한석규 박철 차광수 곽진영 김소이...,첫사랑 수진과 열렬한 연애 끝에 결혼까지 성공했다 남에게 피해 주지 않고 올곧고 바...,감우성 닮으면 잘생긴거냐내 친구 미용실가서 누나가 감우성 닮았다는데 잘생긴거냐 정우...,"괜찮은, 예민함이, 있었네요, 작음, 그러겟지, 자유롭게, 당해서, 있나, 비슷해,..."
1,강경준,39,남,초등학교 시절에는 야구 중학교 시절에는 농구선수였었다 농구를 소재로 한 드라마 마지...,흙수저 아니 몸짱 얼짱 마음까지 짱인 태권도장 사범 얼굴만 봐도 신뢰감이 뚝뚝 묻어...,강경준 강 산 강승호네이름 두번이나 바꿨구나그러고보니 남다 딱 장신영강경준이랑 ...,"이뻐, 괜찮은, 예민함이, 있었네요, 그러면서, 쿨한, 작음, 그러겟지, 자유롭게,..."


### 2) activation_content, 배역내용, 제목, 내용 합쳐서 새로운 columns 생성

In [18]:
# 전체 내용을 담을 list
actor_content_list = []

# 각 columns의 내용이 담긴 리스트 생성
activation_list = actor_df['activation_content'].to_list()
act_list = actor_df['배역내용'].to_list()
community_list = actor_df['제목, 내용'].to_list()

In [19]:
# 합치기
for i in range(len(activation_list)):
    actor_content_list.append(activation_list[i] + act_list[i] + community_list[i])

In [21]:
# 새로운 columns 생성
actor_df['content'] = actor_content_list

In [22]:
# 확인
actor_df.head(2)

,Name,Age,Sex,activation_content,배역내용,"제목, 내용",keywords,content
0,감우성,52,남,기 공채 탤런트로 배우 생활을 시작했으며 동기로 한석규 박철 차광수 곽진영 김소이...,첫사랑 수진과 열렬한 연애 끝에 결혼까지 성공했다 남에게 피해 주지 않고 올곧고 바...,감우성 닮으면 잘생긴거냐내 친구 미용실가서 누나가 감우성 닮았다는데 잘생긴거냐 정우...,"괜찮은, 예민함이, 있었네요, 작음, 그러겟지, 자유롭게, 당해서, 있나, 비슷해,...",기 공채 탤런트로 배우 생활을 시작했으며 동기로 한석규 박철 차광수 곽진영 김소이...
1,강경준,39,남,초등학교 시절에는 야구 중학교 시절에는 농구선수였었다 농구를 소재로 한 드라마 마지...,흙수저 아니 몸짱 얼짱 마음까지 짱인 태권도장 사범 얼굴만 봐도 신뢰감이 뚝뚝 묻어...,강경준 강 산 강승호네이름 두번이나 바꿨구나그러고보니 남다 딱 장신영강경준이랑 ...,"이뻐, 괜찮은, 예민함이, 있었네요, 그러면서, 쿨한, 작음, 그러겟지, 자유롭게,...",초등학교 시절에는 야구 중학교 시절에는 농구선수였었다 농구를 소재로 한 드라마 마지...


In [24]:
# 필요없는 columns를 제거한 DataFrame 생성
actor_df = actor_df[['Name', 'Age', 'Sex', 'content', 'keywords']]

In [27]:
# 확인
actor_df.head(2)

,Name,Age,Sex,content,keywords
0,감우성,52,남,기 공채 탤런트로 배우 생활을 시작했으며 동기로 한석규 박철 차광수 곽진영 김소이...,"괜찮은, 예민함이, 있었네요, 작음, 그러겟지, 자유롭게, 당해서, 있나, 비슷해,..."
1,강경준,39,남,초등학교 시절에는 야구 중학교 시절에는 농구선수였었다 농구를 소재로 한 드라마 마지...,"이뻐, 괜찮은, 예민함이, 있었네요, 그러면서, 쿨한, 작음, 그러겟지, 자유롭게,..."


## 4. 키워드 추출

### 1) 모델 로드

In [26]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

### 2) 임베딩과 키워드 top5 추출

### (1) 배우 키워드 추출

In [29]:
# Nax Sum Similarity 함수 정의
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [words[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [28]:
# 배우 내용과 형용사 가져오기
actor_content_list = actor_df['content'].to_list()
actor_adjective_list = []

for i in actor_df['keywords']:
    actor_adjective_list.append(i.split(', '))

In [34]:
# 배우 키워드 추출
keywords = []

for i in tqdm_notebook(range(len(actor_content_list))):
    # 임베딩
    doc_embedding = model.encode([actor_content_list[i]])
    candidate_embeddings = model.encode(actor_adjective_list[i])
    
    # keyword 추출
    keyword = max_sum_sim(doc_embedding, candidate_embeddings, actor_adjective_list[i], 5, 10)
    keywords.append(", ".join(keyword))

/tmp/ipykernel_4312/622855675.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(actor_content_list))):


  0%|          | 0/252 [00:00<?, ?it/s]

In [35]:
# keyword columns 생성
actor_df['5_keywords'] = keywords
actor_df.head(2)

,Name,Age,Sex,content,keywords,5_keywords
0,감우성,52,남,기 공채 탤런트로 배우 생활을 시작했으며 동기로 한석규 박철 차광수 곽진영 김소이...,"괜찮은, 예민함이, 있었네요, 작음, 그러겟지, 자유롭게, 당해서, 있나, 비슷해,...","성장해, 부답인, 쎄냐고, 좋아하는데, 좋아했는데"
1,강경준,39,남,초등학교 시절에는 야구 중학교 시절에는 농구선수였었다 농구를 소재로 한 드라마 마지...,"이뻐, 괜찮은, 예민함이, 있었네요, 그러면서, 쿨한, 작음, 그러겟지, 자유롭게,...","당해서, 좋아해요, 재미있게, 불쌍해서, 좋아하는데"


In [37]:
# Name, Age, Sex, 5_keywords columns 만 가지는 새로운 데이터 프레임 생성
actor_df = actor_df[['Name', 'Age', 'Sex', '5_keywords']]
actor_df.head(2)

,Name,Age,Sex,5_keywords
0,감우성,52,남,"성장해, 부답인, 쎄냐고, 좋아하는데, 좋아했는데"
1,강경준,39,남,"당해서, 좋아해요, 재미있게, 불쌍해서, 좋아하는데"


In [47]:
# 데이터 프레임 저장
actor_df.to_csv('../text_data/actor_keywords.csv', encoding='utf-8-sig', index=False)

### (2) 웹툰 키워드 추출

In [62]:
# null 값 정리
webtoon_df = webtoon_df.dropna().reset_index(drop=True)

In [63]:
# 웹툰 등장인물 내용과 형용사 가져오기
webtoon_content_list = webtoon_df['설명'].to_list()
webtoon_adjective_list = []

for adjective in webtoon_df['keyword']:
    webtoon_adjective_list.append(adjective.split(', '))

In [68]:
# 웹툰 키워드 추출
keywords = []

for i in tqdm_notebook(range(len(webtoon_content_list))):
    # 임베딩
    doc_embedding = model.encode([webtoon_content_list[i]])
    candidate_embeddings = model.encode(webtoon_adjective_list[i])
    
    # keyword 추출
    try:
        keyword = max_sum_sim(doc_embedding, candidate_embeddings, webtoon_adjective_list[i], 5, 10)
        keywords.append(", ".join(keyword))
    except:
        keywords.append(np.nan)

/tmp/ipykernel_4312/2092822361.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(webtoon_content_list))):


  0%|          | 0/511 [00:00<?, ?it/s]

In [69]:
# keyword columns 생성
webtoon_df['5_keywords'] = keywords
webtoon_df.head(2)

,Title,Character,Sex,Age,설명,keyword,5_keywords
0,폭풍의전학생,주인공,남,10대,최완서를 얼굴에 니킥을 작렬시켜 한방에 보내버리고 정해인을 박치기 한다음에 머리를 ...,"순전히, 있지만, 엄청난, 태연한, 뛰어난데, 가까운, 약해서, 진정한, 안되지만,...","진정한, 당하며, 뛰어난데, 태연한, 안되지만"
1,폭풍의전학생,최완서,남,10대,전 3반짱. 묘한 색깔의 피부[7]와 부서진 콧잔등이 포인트. 전설의 레전드의 최대...,"불쌍한, 멀쩡하던, 수많은, 자리는, 평범하다, 좋아하는, 아니며, 없으며, 강한,...","강한, 좋아하는, 단련해서, 불쌍한, 험하다"


In [71]:
# Title, Character, Sex, Age, 5_keywords columns만 가지는 새로운 데이터 프레임 생성
webtoon_df = webtoon_df[['Title', 'Character', 'Sex', 'Age', '5_keywords']]
webtoon_df.head(2)

,Title,Character,Sex,Age,5_keywords
0,폭풍의전학생,주인공,남,10대,"진정한, 당하며, 뛰어난데, 태연한, 안되지만"
1,폭풍의전학생,최완서,남,10대,"강한, 좋아하는, 단련해서, 불쌍한, 험하다"


In [72]:
# 데이터 프레임 저장
webtoon_df.to_csv('../text_data/webtoon_keywords.csv', encoding='utf-8-sig', index=False)